<a href="https://colab.research.google.com/github/nforesperance/TensorFlow/blob/master/retrain_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install pyyaml h5py  # Required to save models in HDF5 format

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense

print(tf.version.VERSION)

2.1.0


In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = (train_images.reshape((train_images.shape[0], 28, 28, 1)))/ 255.0
test_images = (test_images.reshape((test_images.shape[0], 28, 28, 1))) / 255.0
test_images = test_images / 255.0
train_images = train_images / 255.0

In [0]:
# Define a simple sequential model
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  model.add(MaxPool2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model
# Create a basic model instance
model = create_model()

In [0]:
checkpoint_path = "trained0/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
save_path = "trained0"

## N:B
###The following cell should be ran only once or model will retrain completely
#### checkpoint is necessary if to be able to load best weights if there was an interuption before restore_best_weight

In [0]:

#N:B The Followind Code is ran only once

# Create a callback that saves the model's weights
easystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='min', baseline=None, restore_best_weights=True)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                            monitor='val_loss',
                            verbose=1,
                            save_weights_only=True,
                            save_best_only=True,
                            mode='min')

# Train the model with the new callback
model.fit(train_images, train_labels, validation_data=(train_images,
                                                       train_labels), 
                                                       epochs=20, 
                                                       batch_size=50, 
                                                       verbose = 2,
                                                       callbacks=[checkpoint,easystopping])
# model.load_weights(checkpoint_path)
# model.save("trained1/best.h5")

In [0]:
model.save(save_path+"/best.h5")

## Retraining
### The following is used to load the trained model and retrain it


In [0]:
for i in range (2):
  del model
  model = tf.keras.models.load_model(save_path+'/best.h5')  
  easystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='min', baseline=None, restore_best_weights=True)

  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                            monitor='val_loss',
                            verbose=1,
                            save_weights_only=True,
                            save_best_only=True,
                            mode='min')

  # Train the model with the new callback
  model.fit(train_images, train_labels, validation_data=(train_images,
                                                       train_labels), 
                                                       epochs=20, 
                                                       batch_size=50, 
                                                       verbose = 2,
                                                       callbacks=[checkpoint,easystopping])
  model.save(save_path+"/best.h5")


## Testing

In [50]:
# model.save("trained/best.h5")
del model
model = tf.keras.models.load_model(save_path+'/best.h5')
# Re-evaluate the model
loss,acc1 = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc1))

del model
model = create_model()
model.load_weights(checkpoint_path)
# Re-evaluate the model
loss,acc2 = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc2))
if acc2 > acc1:
  model.save(save_path+"/best.h5")
  print("Checkpoint Model Used")


10000/10000 - 1s - loss: 1.4761 - accuracy: 0.9852
Restored model, accuracy: 98.52%
10000/10000 - 1s - loss: 1.4761 - accuracy: 0.9852
Restored model, accuracy: 98.52%
